# GPU Information

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
# memory footprint support libraries/code

!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm() 

In [ ]:
#!kill -9 -1

# Data Warehousing Setup



In [ ]:
# Mount Google Drive folder

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Add models repos

%cd  /content/drive/My Drive/Models Running

In [ ]:
#! rm -rf BertSum

In [ ]:
! git clone https://github.com/yannistze/BertSum.git

In [ ]:
# Upload files into shared Google Drive folder and unzip them

%cd /content/drive/My Drive/Models Running/BertSum/bert_data

from google.colab import files
files.upload()

%cd /content/drive/My Drive/Models Running/BertSum/bert_data/
! unzip bertsum_data.zip

%cd /content/drive/My Drive/Models Running/BertSum

# BertSum Model


In [ ]:
# Install Needed Packages

! pip install pytorch_pretrained_bert
! pip install tensorboardX
! pip install pyrouge
! pip install multiprocess 

In [ ]:
# Move to source folder in order to run BertSum
%cd /content/drive/My Drive/Models Running/BertSum/src

In [ ]:
# Step 1 - Train the BertSum Classifier Layer

! python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_classifier -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -decay_method noam -train_steps 25000 -accum_count 2 -log_file ../logs/bert_classifier -use_interval true -warmup_steps 10000

In [ ]:
# Step 1 - Continue Training from previous training savepoint for the BertSum Classifier Layer

! python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_classifier -train_from ../models/bert_classifier/model_step_20000.pt -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -decay_method noam -train_steps 25000 -accum_count 2 -log_file ../logs/bert_classifier -use_interval true -warmup_steps 10000

In [ ]:
# Step 2 - Train the BertSum Transformer Layer

! python train.py -mode train -encoder transformer -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_transformer -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -decay_method noam -train_steps 25000 -accum_count 2 -log_file ../logs/bert_transformer -use_interval true -warmup_steps 10000 -ff_size 2048 -inter_layers 2 -heads 8

In [ ]:
# Step 3 - Train the BertSum RNN Layer

! python train.py -mode train -encoder rnn -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_rnn -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -decay_method noam -train_steps 25000 -accum_count 2 -log_file ../logs/bert_rnn -use_interval true -warmup_steps 10000 -rnn_size 768 -dropout 0.1

In [ ]:
# Step 3 - Continue Training from previous training savepoint for the BertSum RNN Layer

! python train.py -mode train -encoder rnn -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_rnn -train_from ../models/bert_rnn/model_step_17000.pt -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -decay_method noam -train_steps 25000 -accum_count 2 -log_file ../logs/bert_rnn -use_interval true -warmup_steps 10000 -rnn_size 768 -dropout 0.1

# Producing the Summaries by Class

In [ ]:
# Producing the summaries for the specied class and layer of BertSum (example: 10000, rnn)
! python train.py -mode test -bert_data_path ~/BertSum/raw_data/genre_bert/10000/ -model_path ~/BertSum/models/bert_rnn -visible_gpus -1 -result_path ~/BertSum/results -block_trigram true -test_from ~/BertSum/models/bert_rnn/model_step_25000.pt

# Save the Summaries to the bucket
! gsutil cp -r ~/BertSum/results/10000/* gs://bertsum-bucket/bert_summary/10000

# Troubleshooting Section (in case of need)

In [ ]:
# Pyrouge troubleshooting in case it is need
! pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
! pip install pyrouge
! pip show pyrouge

! python -m pyrouge.test

# Set the Environment Variable to point to the correct folder
!pyrouge_set_rouge_path '/content/drive/My Drive/Models Running/BertSum/src/pyrouge/tools/ROUGE-1.5.5'

# ROUGE troubleshooting in case it is need
! chmod 777 '/content/drive/My Drive/Models Running/BertSum/src/pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl'

%cd '/content/drive/My Drive/Models Running/BertSum/src/pyrouge/tools/ROUGE-1.5.5/'
!perl ROUGE-1.5.5.pl

%cd '/content/drive/My Drive/Models Running/BertSum/src/pyrouge/tools/ROUGE-1.5.5/data/'
! perl  ./WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

! sudo apt-get install libxml-parser-perl